In [1]:
import glob
import numpy as np
import tensorflow as tf
import cv2
import json
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_dir = '/content/drive/Othercomputers/내 컴퓨터/label'
train_json = glob.glob(train_dir + '/*.*')

In [5]:
len(train_json)

18517

Val 1 미세각질
Val 2 피지과다
Val 3 모낭사이홍반
Val 4 모낭홍반/농포
Val 5 비듬
Val 6 탈모

각각 4가지 상태가 있는데, 이를 모두 고려한다면 4096 가지의 classification을 진행해야함. 따라서 각 상태에 대해서 증상 유, 무만 고려할 예정.

In [6]:
status = ['value_1', 'value_2', 'value_3', 'value_4', 'value_5', 'value_6']
with open(train_json[0]) as j:
    example = json.load(j)
    
example

{'image_id': '9864_A2LEBJJDE001167_1602913005874_5',
 'image_file_name': '9864_A2LEBJJDE001167_1602913005874_5_RH.jpg',
 'value_1': '1',
 'value_2': '0',
 'value_3': '0',
 'value_4': '1',
 'value_5': '0',
 'value_6': '3'}

아래 코드는 전처리 코드. 개인 컴퓨터에서 진행하여 colab에선 실행 불가능.

In [ ]:
# for name in train_json[10:]:
#     with open(name, 'r',encoding='utf-8') as j:
#         data = json.load(j)
#         f_name = data['image_file_name']
#         # print(train_dir+ '/' + f_name)
#         img = cv2.imread(train_dir+ '/' + f_name, cv2.IMREAD_COLOR)
#         # print(img)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         img = img[80:560,:]
#         img = cv2.resize(img, (64,64))
#         cv2.imwrite('Training/'+f_name,img)       

In [ ]:
# np.save('/content/drive/MyDrive/skku_ai/x_train', x_train)
# np.save('/content/drive/MyDrive/skku_ai/y_train_1', y_train_1)
# np.save('/content/drive/MyDrive/skku_ai/y_train_2', y_train_2)
# np.save('/content/drive/MyDrive/skku_ai/y_train_3', y_train_3)
# np.save('/content/drive/MyDrive/skku_ai/y_train_4', y_train_4)
# np.save('/content/drive/MyDrive/skku_ai/y_train_5', y_train_5)
# np.save('/content/drive/MyDrive/skku_ai/y_train_6', y_train_6)

In [7]:

x, y = [],[[],[],[],[],[],[],[]]
for name in train_json:
  try:
    with open(name, 'r',encoding='utf-8') as j:
        data = json.load(j)
        f_name = data['image_file_name']
        f = '/content/drive/Othercomputers/내 컴퓨터/img/'+f_name
        # print(f)
        # img = cv2.imread(f, cv2.IMREAD_COLOR)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = tf.io.read_file(f)
        img = img.numpy()
        # for idx, n in enumerate(status):
        #     i = idx + 1

        #     if data[n] == '0':
        #         y[i].append(0)
        #     else:
        #         y[i].append(1)
        if data['value_6'] == '0':
          y[6].append(0)
        else:
          y[6].append(1)
        x.append(img)
  except UnicodeDecodeError: continue

KeyboardInterrupt: ignored

In [8]:
len(x)

5581

In [9]:
dump, y_1, y_2, y_3,y_4,y_5,y_6 = y
x = np.array(x)
# x = x/255

y_1 = np.array(y_1).reshape(-1,1)
y_2 = np.array(y_2).reshape(-1,1)
y_3 = np.array(y_3).reshape(-1,1)
y_4 = np.array(y_4).reshape(-1,1)
y_5 = np.array(y_5).reshape(-1,1)
y_6 = np.array(y_6).reshape(-1,1)



In [10]:
np.save("/content/drive/MyDrive/skku_ai/x", x)

In [11]:
x_train, x_test = train_test_split(x, test_size=0.2,random_state=123)
y_train_1, y_test_1 = train_test_split(y_6, test_size=0.2,random_state=123)

In [12]:
model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3),
    classes=2,
    classifier_activation='softmax'
)
model.summary()

16705208/16705208 [==============================] - 3s 0us/step
Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TF

In [15]:
saved_model = '/content/drive/MyDrive/skku_ai/test1.h5'
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25,restore_best_weights=True)
mc = tf.keras.callbacks.ModelCheckpoint(saved_model, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
               
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

history = model.fit(x_train, y_train_1,
        epochs = 100,
        callbacks=[es, mc],
        batch_size = 32,
        validation_data = (x_test, y_test_1))

Epoch 1/100


ValueError: ignored

In [ ]:

# model1 = tf.keras.models.Sequential()
# #
# model1.add(tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',input_shape=(224,224,3)))  # conv2d 수행하면 64채널의 결과가 나오고, kernelsize = 3
# model1.add(tf.keras.layers.Dropout(rate=0.5))                          #  DO1
# model1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# #
# model1.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
# model1.add(tf.keras.layers.Dropout(rate=0.5))                          # DO2
# model1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# #
# model1.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))
# model1.add(tf.keras.layers.Dropout(rate=0.5))                           # DO3
# model1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# #
# model1.add(tf.keras.layers.Flatten())
# model1.add(tf.keras.layers.Dense(2, activation='softmax'))
# #
# model1.summary()

# model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
               
#                loss='sparse_categorical_crossentropy',
#                metrics=['accuracy'])